In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import time
from playsound import playsound
import joblib
import threading

# === Load the model ===
model = load_model("model/mask_detection_cnn_final.h5")

# === Load Label Encoder using joblib ===
label_encoder = joblib.load("model/label_encoder_cnn.pkl")

# === Audio files path ===
READY_SOUND = "audio/ready.wav"
WITH_MASK_SOUND = "audio/with_mask.wav"
WITHOUT_MASK_SOUND = "audio/without_mask.wav"

# === Function to play sound in a thread ===
def play_sound(path):
    threading.Thread(target=playsound, args=(path,), daemon=True).start()

# === Load Haar Cascade for face detection ===
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# === Start webcam ===
cap = cv2.VideoCapture(0)
print("Camera ready. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        face_img_resized = cv2.resize(face_img, (224, 224))
        face_input = np.expand_dims(face_img_resized, axis=0) / 255.0

        prediction = model.predict(face_input)
        class_index = np.argmax(prediction)
        label = label_encoder.inverse_transform([class_index])[0]

        # Draw rectangle and label
        color = (0, 255, 0) if label == "with_mask" else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

        # Play audio feedback in background thread
        play_sound(READY_SOUND)
        if label == "with_mask":
            play_sound(WITH_MASK_SOUND)
        else:
            play_sound(WITHOUT_MASK_SOUND)

        time.sleep(2)  # Delay to avoid rapid repeat alerts

    cv2.imshow("Face Mask Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


C:\Users\divya\anaconda3\envs\tfenv\lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Camera ready. Press 'q' to quit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step


In [14]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# === Load trained model and label encoder ===
model = load_model("model/mask_detection_cnn_final.h5")

label_map = {
    0: "with_mask",
    1: "without_mask"
}
# === Load Haar Cascade for face detection ===
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# === Start webcam ===
cap = cv2.VideoCapture(0)

print("📷 Camera started. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to read from camera.")
        break

    frame = cv2.flip(frame, 1)  # Mirror effect
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        face_img_resized = cv2.resize(face_img, (224, 224))  # Make sure your model uses 224x224 input
        face_input = np.expand_dims(face_img_resized, axis=0) / 255.0  # Normalize

        prediction = model.predict(face_input, verbose=0)
        class_index = np.argmax(prediction)
        label = label_map.get(class_index, "Unknown")


        color = (0, 255, 0) if label == "with_mask" else (0, 0, 255)
        label_text = "With Mask" if label == "with_mask" else "No Mask"

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label_text, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("Mask Detection Live Feed", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


📷 Camera started. Press 'q' to quit.


In [15]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# === Load trained model ===
model = load_model("model/mask_detection_cnn_final.h5")

# === Label encoder using dictionary ===
label_map = {
    1: "with_mask",
    0: "without_mask"
}

# === Load Haar Cascades (frontal + profile) ===
frontal_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
profile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_profileface.xml")

# === Start webcam ===
cap = cv2.VideoCapture(0)

print("📷 Camera started. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to read from camera.")
        break

    frame = cv2.flip(frame, 1)  # Mirror effect
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect face: try frontal first, then profile if not found
    faces = frontal_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    if len(faces) == 0:
        faces = profile_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    if len(faces) == 0:
        cv2.putText(frame, "No face detected", (30, 30), cv2.FONT_HERSHEY_SIMPLEX,
                    0.8, (0, 255, 255), 2)
    else:
        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            face_img_resized = cv2.resize(face_img, (224, 224))  # Match model input size
            face_input = np.expand_dims(face_img_resized, axis=0) / 255.0  # Normalize

            prediction = model.predict(face_input, verbose=0)
            class_index = np.argmax(prediction)
            label = label_map.get(class_index, "Unknown")

            # Drawing logic
            color = (0, 255, 0) if label == "with_mask" else (0, 0, 255)
            label_text = "With Mask" if label == "with_mask" else "No Mask"

            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, label_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.9, color, 2)

    # Show the frame
    cv2.imshow("Mask Detection Live Feed", frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


📷 Camera started. Press 'q' to quit.


In [16]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# === Load trained model ===
model = load_model("model/mask_detection_cnn_final.h5")

# === Corrected label map ===
label_map = {
    0: "with_mask",       # class 0: with mask
    1: "without_mask"     # class 1: without mask
}

# === Load Haar Cascade for face detection ===
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# === Start webcam ===
cap = cv2.VideoCapture(0)
print("📷 Camera started. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to read from camera.")
        break

    frame = cv2.flip(frame, 1)  # Mirror effect
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        face_img_resized = cv2.resize(face_img, (224, 224))  # Change to (128,128) if needed
        face_input = np.expand_dims(face_img_resized, axis=0) / 255.0  # Normalize

        prediction = model.predict(face_input, verbose=0)
        class_index = np.argmax(prediction)
        label = label_map.get(class_index, "Unknown")

        # Print raw prediction and debug info
        print(f"Prediction: {prediction}, Class Index: {class_index}, Label: {label}")

        # Set color and label
        color = (0, 255, 0) if label == "with_mask" else (0, 0, 255)
        label_text = "With Mask" if label == "with_mask" else "No Mask"

        # Confidence score
        confidence = prediction[0][class_index]
        display_text = f"{label_text} ({confidence:.2f})"

        # Draw rectangle and label
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, display_text, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Mask Detection Live Feed", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


📷 Camera started. Press 'q' to quit.
Prediction: [[0.9595739]], Class Index: 0, Label: with_mask
Prediction: [[0.86309034]], Class Index: 0, Label: with_mask
Prediction: [[0.9439301]], Class Index: 0, Label: with_mask
Prediction: [[0.9078469]], Class Index: 0, Label: with_mask
Prediction: [[0.92952156]], Class Index: 0, Label: with_mask
Prediction: [[0.9484013]], Class Index: 0, Label: with_mask
Prediction: [[0.89172614]], Class Index: 0, Label: with_mask
Prediction: [[0.77708554]], Class Index: 0, Label: with_mask
Prediction: [[0.86609954]], Class Index: 0, Label: with_mask
Prediction: [[0.9366469]], Class Index: 0, Label: with_mask
Prediction: [[0.8831303]], Class Index: 0, Label: with_mask
Prediction: [[0.83791435]], Class Index: 0, Label: with_mask
Prediction: [[0.9290387]], Class Index: 0, Label: with_mask
Prediction: [[0.9042605]], Class Index: 0, Label: with_mask
Prediction: [[0.8891673]], Class Index: 0, Label: with_mask
Prediction: [[0.93412745]], Class Index: 0, Label: with_m

In [17]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# === Load trained model ===
model = load_model("model/mask_detection_cnn_final.h5")

# === Corrected label map ===
label_map = {
    0: "without_mask",      # class 0: without mask
    1: "with_mask"          # class 1: with mask
}

# === Load Haar Cascade for face detection ===
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# === Start webcam ===
cap = cv2.VideoCapture(0)
print("📷 Camera started. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to read from camera.")
        break

    frame = cv2.flip(frame, 1)  # Mirror effect
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        face_img_resized = cv2.resize(face_img, (224, 224))
        face_input = np.expand_dims(face_img_resized, axis=0) / 255.0  # Normalize

        prediction = model.predict(face_input, verbose=0)
        class_index = np.argmax(prediction)
        confidence = prediction[0][class_index]
        label = label_map.get(class_index, "Unknown")

        print(f"🧠 Prediction: {prediction}, Class Index: {class_index}, Label: {label}")

        # === Set color based on label ===
        color = (0, 255, 0) if label == "with_mask" else (0, 0, 255)
        label_text = "With Mask" if label == "with_mask" else "No Mask"
        display_text = f"{label_text} ({confidence:.2f})"

        # === Draw bounding box and label ===
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, display_text, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Mask Detection Live Feed", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


📷 Camera started. Press 'q' to quit.
🧠 Prediction: [[0.84767735]], Class Index: 0, Label: without_mask
🧠 Prediction: [[0.52731913]], Class Index: 0, Label: without_mask
🧠 Prediction: [[0.7046289]], Class Index: 0, Label: without_mask
🧠 Prediction: [[0.84077805]], Class Index: 0, Label: without_mask
🧠 Prediction: [[0.8187837]], Class Index: 0, Label: without_mask
🧠 Prediction: [[0.7142362]], Class Index: 0, Label: without_mask
🧠 Prediction: [[0.52749324]], Class Index: 0, Label: without_mask
🧠 Prediction: [[0.8210535]], Class Index: 0, Label: without_mask
🧠 Prediction: [[0.7455883]], Class Index: 0, Label: without_mask
🧠 Prediction: [[0.8913747]], Class Index: 0, Label: without_mask
🧠 Prediction: [[0.8302098]], Class Index: 0, Label: without_mask
🧠 Prediction: [[0.47880027]], Class Index: 0, Label: without_mask
🧠 Prediction: [[0.5998082]], Class Index: 0, Label: without_mask
🧠 Prediction: [[0.6250346]], Class Index: 0, Label: without_mask
🧠 Prediction: [[0.7470155]], Class Index: 0, Lab

In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# === Load trained model ===
model = load_model("model/mask_detection_cnn_final.h5")

# === Label map for binary classification ===
label_map = {
    1: "without_mask",
    0: "with_mask"
}

# === Haar Cascade for face detection ===
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# === Start webcam ===
cap = cv2.VideoCapture(0)
print("📷 Camera started. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to read from camera.")
        break

    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        face_img_resized = cv2.resize(face_img, (224, 224))
        face_input = np.expand_dims(face_img_resized, axis=0) / 255.0

        prediction = model.predict(face_input, verbose=0)
        prob = prediction[0][0]

        # Fix: convert sigmoid output to class index
        class_index = 1 if prob > 0.5 else 0
        label = label_map[class_index]

        print(f"🧠 Prediction: {prob:.4f}, Class Index: {class_index}, Label: {label}")

        color = (0, 255, 0) if class_index == 1 else (0, 0, 255)
        label_text = "With Mask" if class_index == 1 else "No Mask"
        display_text = f"{label_text} ({prob:.2f})"

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, display_text, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Mask Detection Live Feed", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


📷 Camera started. Press 'q' to quit.
🧠 Prediction: 0.9624, Class Index: 1, Label: without_mask
🧠 Prediction: 0.9763, Class Index: 1, Label: without_mask
🧠 Prediction: 0.9630, Class Index: 1, Label: without_mask
🧠 Prediction: 0.9524, Class Index: 1, Label: without_mask
🧠 Prediction: 0.6723, Class Index: 1, Label: without_mask
🧠 Prediction: 0.7552, Class Index: 1, Label: without_mask
🧠 Prediction: 0.9841, Class Index: 1, Label: without_mask
🧠 Prediction: 0.9841, Class Index: 1, Label: without_mask
🧠 Prediction: 0.9786, Class Index: 1, Label: without_mask
🧠 Prediction: 0.9818, Class Index: 1, Label: without_mask
🧠 Prediction: 0.9077, Class Index: 1, Label: without_mask
🧠 Prediction: 0.4241, Class Index: 0, Label: with_mask
🧠 Prediction: 0.5618, Class Index: 1, Label: without_mask
🧠 Prediction: 0.8796, Class Index: 1, Label: without_mask
🧠 Prediction: 0.7808, Class Index: 1, Label: without_mask
🧠 Prediction: 0.8926, Class Index: 1, Label: without_mask
🧠 Prediction: 0.7398, Class Index: 1, 

In [3]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import os

# === Load trained CNN model ===
model = load_model("model/mask_detection_cnn_final.h5")

# === Try loading label encoder ===
try:
    with open("model/label_encoder_cnn.pkl", "rb") as f:
        label_encoder = pickle.load(f)
        label_map = {i: label for i, label in enumerate(label_encoder.classes_)}
        print("✅ Label encoder loaded.")
except Exception as e:
    print("⚠️ Failed to load label encoder. Using fallback label map.")
    label_map = {
        0: "with_mask",
        1: "without_mask"
    }

# === Load Haar Cascade for face detection ===
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# === Start webcam ===
cap = cv2.VideoCapture(0)
print("📷 Camera started. Press 's' to scan, 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to read from camera.")
        break

    frame = cv2.flip(frame, 1)  # Mirror effect
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Draw rectangle around face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 0), 2)
        cv2.putText(frame, "Press 's' to Scan", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

    cv2.imshow("📷 Mask Detection - Press 's' to Scan", frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('s') and len(faces) > 0:
        print("📸 Scanning face...")
        for (x, y, w, h) in faces:
            face_img = frame[y:y + h, x:x + w]
            face_resized = cv2.resize(face_img, (224, 224))
            face_input = np.expand_dims(face_resized, axis=0) / 255.0

            prediction = model.predict(face_input, verbose=0)
            class_index = np.argmax(prediction)
            confidence = prediction[0][class_index]
            label = label_map.get(class_index, "Unknown")

            # Show prediction result
            print(f"🧠 Prediction: {prediction}")
            print(f"📊 Confidence: {confidence:.2f}")
            print(f"🧾 Result: {label.upper()}")

            # Display result on frame
            result_text = f"{label.upper()} ({confidence:.2f})"
            color = (0, 255, 0) if label == "with_mask" else (0, 0, 255)
            cv2.putText(frame, result_text, (x, y - 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 3)

            cv2.imshow("📷 Mask Detection Result", frame)
            cv2.waitKey(3000)  # Show for 3 seconds

    elif key == ord('q'):
        print("👋 Exiting.")
        break

cap.release()
cv2.destroyAllWindows()


⚠️ Failed to load label encoder. Using fallback label map.
📷 Camera started. Press 's' to scan, 'q' to quit.
📸 Scanning face...
🧠 Prediction: [[0.97168416]]
📊 Confidence: 0.97
🧾 Result: WITH_MASK
🧠 Prediction: [[0.20183451]]
📊 Confidence: 0.20
🧾 Result: WITH_MASK
📸 Scanning face...
🧠 Prediction: [[0.9767584]]
📊 Confidence: 0.98
🧾 Result: WITH_MASK
👋 Exiting.


In [4]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# === Load trained model ===
model = load_model("model/mask_detection_cnn_final.h5")

# === Label map for binary classification ===
label_map = {
    1: "without_mask",
    0: "with_mask"
}

# === Haar Cascade for face detection ===
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# === Start webcam ===
cap = cv2.VideoCapture(0)
print("📷 Camera started. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to read from camera.")
        break

    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        face_img_resized = cv2.resize(face_img, (224, 224))
        face_input = np.expand_dims(face_img_resized, axis=0) / 255.0

        prediction = model.predict(face_input, verbose=0)
        prob = prediction[0][0]

        class_index = 1 if prob > 0.5 else 0
        label = label_map[class_index]

        color = (0, 255, 0) if class_index == 0 else (0, 0, 255)
        label_text = "With Mask" if class_index == 0 else "No Mask"

        # Draw rectangle and label (without accuracy value)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label_text, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Mask Detection Live Feed", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


📷 Camera started. Press 'q' to quit.


In [5]:
import cv2
import numpy as np
import threading
from playsound import playsound
from tensorflow.keras.models import load_model

# === Load trained model ===
model = load_model("model/mask_detection_cnn_final.h5")

# === Label map for binary classification ===
# 0 → with_mask, 1 → without_mask
label_map = {
    0: "with_mask",
    1: "without_mask"
}

# === Paths to your audio files ===
AUDIO_FILES = {
    "with_mask": "audio/with_mask.wav",
    "without_mask": "audio/without_mask.wav"
}

def play_audio_async(path):
    """Play audio in a separate thread (non‑blocking)."""
    threading.Thread(target=playsound, args=(path,), daemon=True).start()

# === Haar Cascade for face detection ===
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# === Start webcam ===
cap = cv2.VideoCapture(0)
print("📷 Camera started. Press 'q' to quit.")

last_label = None

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to read from camera.")
        break

    frame = cv2.flip(frame, 1)
    gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        # extract and prepare face ROI
        face_roi = frame[y:y+h, x:x+w]
        face_resized = cv2.resize(face_roi, (224, 224))
        face_input   = np.expand_dims(face_resized, axis=0) / 255.0

        # model prediction (sigmoid)
        prob = model.predict(face_input, verbose=0)[0][0]
        class_index = 1 if prob > 0.5 else 0
        label = label_map[class_index]

        # only play audio when label changes
        if label != last_label:
            play_audio_async(AUDIO_FILES[label])
            last_label = label

        # draw bounding box + label
        color = (0,255,0) if class_index==0 else (0,0,255)
        text  = "With Mask" if class_index==0 else "No Mask"
        cv2.rectangle(frame, (x,y), (x+w,y+h), color, 2)
        cv2.putText(frame, text, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Mask Detection Live Feed", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


📷 Camera started. Press 'q' to quit.


In [2]:
import cv2
import numpy as np
import threading
import time
from playsound import playsound
from tensorflow.keras.models import load_model

# === Load trained model ===
model = load_model("model/mask_detection_cnn_final.h5")

# === Label map for binary classification ===
label_map = {
    0: "with_mask",
    1: "without_mask"
}

# === Paths to your audio files ===
AUDIO_FILES = {
    "with_mask": "audio/with_mask.wav",
    "without_mask": "audio/without_mask.wav"
}

def play_audio_async(path):
    threading.Thread(target=playsound, args=(path,), daemon=True).start()

# === Haar Cascade for face detection ===
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# === Start webcam ===
cap = cv2.VideoCapture(0)
print("📷 Camera started. Press 'q' to quit.")

last_label = None
last_audio_time = 0
COOLDOWN = 3.0  # seconds

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to read from camera.")
        break

    frame = cv2.flip(frame, 1)
    gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi = frame[y:y+h, x:x+w]
        face = cv2.resize(roi, (224, 224))
        inp  = np.expand_dims(face, axis=0) / 255.0

        prob = model.predict(inp, verbose=0)[0][0]
        cls  = 1 if prob>0.5 else 0
        label= label_map[cls]

        now = time.time()
        # only play if label changed AND cooldown expired
        if label != last_label and (now - last_audio_time) >= COOLDOWN:
            play_audio_async(AUDIO_FILES[label])
            last_label = label
            last_audio_time = now

        # draw box + label
        color = (0,255,0) if cls==0 else (0,0,255)
        text  = "With Mask" if cls==0 else "No Mask"
        cv2.rectangle(frame, (x,y),(x+w,y+h), color, 2)
        cv2.putText(frame, text, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Mask Detection Live Feed", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


📷 Camera started. Press 'q' to quit.
